source = https://medium.com/p/5c68e8442df1

In [47]:
import pandas as pd
import numpy as np
import torch
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import time


In [40]:

pathname = '/Users/tomcio/Documents/GitHub/my-own-semantic-search-engine/data/'

df = pd.read_csv(pathname + 'hotel_reviews.csv')

In [41]:
data = df[['reviews.text']]
data

,reviews.text
0,Our experience at Rancho Valencia was absolute...
1,Amazing place. Everyone was extremely warm and...
2,We booked a 3 night stay at Rancho Valencia to...
3,Currently in bed writing this for the past hr ...
4,I live in Md and the Aloft is my Home away fro...
...,...
9995,It is hard for me to review an oceanfront hote...
9996,"I live close by, and needed to stay somewhere ..."
9997,Rolled in 11:30 laid out heads down woke up to...
9998,Absolutely terrible..I was told I was being gi...


In [42]:
#Load a pre-trained model
model =SentenceTransformer('msmarco-MiniLM-L-12-v3')
hotel_reviews=data["reviews.text"].tolist()

#embed hotel reviews
hotel_reviews_embds=model.encode(hotel_reviews)

In [43]:
# Create an index using FAISS
index = faiss.IndexFlatL2(hotel_reviews_embds.shape[1])
index.add(hotel_reviews_embds)
faiss.write_index(index, 'index_hotel_reviews')
index = faiss.read_index('index_hotel_reviews')

In [44]:
def search(query):
    
    t=time.time()
    query_vector = model.encode([query])
    k = 5
    top_k = index.search(query_vector, k)
    print('totaltime: {}'.format(time.time()-t))
    return [hotel_reviews[_id] for _id in top_k[1].tolist()[0]]

In [45]:
results=search("slow service")
results

totaltime: 0.019793272018432617


["1. I did not like the experience of dealing with the Expedia person who assisted with the reservation. 2. It was annoying to learn only while checking out that this is a time share hotel. No one told me. 3. The WIFI was slow, and unreliable, cutting out from time to time. Also, when it tried to upgrade to the offered higher speed WIFI, it was quite an ordeal that took about 90 minutes and a call to the help line at your WIFI vendor. BTW, you might try the user interface for that sign up - it exasperatingly erases all one's data entries if a single one of them is incorrect (eg if you generate an unacceptable password) 4. The Business Center has two PCs that are excruciatingly slow, and a printer as well that lumbers along. Old, slow, out of date. 5. The single bright light in this business experience is that the staff were unfailingly polite and did their best to help. I give them credit for trying, and for being nice to me.",
 'Bad and slow WiFi connection',
 'The internet was so slo